 ─────────────────────────────────────────────────────────────
 
 Notebook: init_block_volume.ipynb
 
 Purpose: Provision persistent block storage (10 GB) and
     attach + mount it to the running Chameleon instance.
 
 Site: CHI@TACC (or any other site with volume support)
 
 Dependencies: python-chi, openstack CLI, root access
 ─────────────────────────────────────────────────────────────

In [ ]:
import os
import time
from chameleon import auth, lease, project, instance, volume

In [ ]:
# Step 1: Authenticate and connect to CHI@TACC
site = "CHI@TACC"  # change if needed
conn = auth.session(site)
project_id = project.project_id(site)
print(f"Connected to {site} | Project ID: {project_id}")

In [ ]:
# Step 2: Create a persistent volume (10GB)
volume_name = "block-project38"
new_vol = volume.create(
    conn,
    name=volume_name,
    size=10,  # in GB
    project_id=project_id
)
print(f"Volume created: {new_vol.id} | Size: {new_vol.size} GB")


In [ ]:
# Step 3: List running instances
instances = instance.list(conn)
for i in instances:
    print(f"Instance: {i.name} | ID: {i.id} | Status: {i.status}")

In [ ]:
# Step 4: Attach the volume to your active instance
instance_id = instances[0].id  # pick the correct one if more than 1
conn.block_storage.attach_volume(instance_id, new_vol.id, device="/dev/vdb")
print(f"Volume attached to instance: {instance_id}")

# Step 5: Format and mount it (run on the instance itself)
Now SSH into your instance

Run:
    
    sudo mkfs.ext4 /dev/vdb

    sudo mkdir -p /mnt/block

    sudo mount /dev/vdb /mnt/block

    sudo chown -R cc:cc /mnt/block

# Create app-specific directories
    
    mkdir -p /mnt/block/postgres_data

    mkdir -p /mnt/block/minio_data

    mkdir -p /mnt/block/mlflow_artifacts

    lsblk -f
